In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



In [2]:

import torch
print(f"GPU Available: {torch.cuda.is_available()}")
print(f"GPU Name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU'}")
print(f"PyTorch Version: {torch.__version__}")


GPU Available: False
GPU Name: No GPU
PyTorch Version: 2.8.0+cu126


In [3]:
# Install required packages
!pip install -q transformers 

print("✅ Installation complete!")

✅ Installation complete!


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# ========================================
# ADD YOUR HUGGING FACE TOKEN HERE
# ========================================
HF_TOKEN = "##"  # Replace with your actual token

MODEL_NAME = "google/medgemma-1.5-4b-it"

print("🔄 Loading {MODEL_NAME}... (this takes 2-5 minutes)")

# Load tokenizer with token
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    token=HF_TOKEN  # <-- Added token parameter
)

# Load model with token and 8-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    dtype=torch.float16,
    device_map="auto",
    token=HF_TOKEN  # <-- Added token parameter
)

print("✅ MedGemma loaded successfully!")
print(f"Model size: {MODEL_NAME}")
print(f"Device: {next(model.parameters()).device}")

🔄 Loading {MODEL_NAME}... (this takes 2-5 minutes)


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

✅ MedGemma loaded successfully!
Model size: google/medgemma-1.5-4b-it
Device: cpu


In [14]:
def generate_response(prompt, max_length=256):
    """
    Generate response with proper chat formatting for instruction models
    """
    # Format as instruction following
    formatted_prompt = f"{prompt}\n\nAssistant:"
    
    inputs = tokenizer(
        formatted_prompt, 
        return_tensors="pt",
        return_attention_mask=True
    ).to(model.device)
    
    # Generate NEW tokens only
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            do_sample=False,  # Deterministic
            pad_token_id=tokenizer.pad_token_id if tokenizer.pad_token_id else tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # Decode ONLY the new tokens (not the input)
    input_length = inputs.input_ids.shape[1]
    generated_tokens = outputs[0][input_length:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    return response.strip()

print("✅ Properly formatted generation function created!")

✅ Properly formatted generation function created!


In [15]:
# Simpler test first
# Test the fixed version
print("🧪 Testing NEW generation...\n")

test_prompt = "What is the UPDRS Part III motor score for a patient with bilateral tremor and moderate bradykinesia? Provide only the numeric score."

response = generate_response(test_prompt, max_length=50)

print(f"Prompt: {test_prompt[:80]}...")
print(f"\n✅ Generated response: '{response}'")
print(f"Length: {len(response)}")

if response and len(response) > 0 and response != test_prompt:
    print("\n🎉 SUCCESS! Model is generating new content!")
else:
    print("\n⚠️ Still not generating properly")

🧪 Testing NEW generation...

Prompt: What is the UPDRS Part III motor score for a patient with bilateral tremor and m...

✅ Generated response: ''
Length: 0

⚠️ Still not generating properly


In [7]:
# Synthetic clinical notes with UPDRS scores
synthetic_data = [
    {
        "note": "Patient presents with mild resting tremor in right hand, amplitude approximately 1cm. No tremor in left hand. Slight rigidity noted in right arm. Finger taps show mild slowing. Gait is normal with good arm swing bilaterally. No postural instability.",
        "updrs_motor": 8,
        "severity": "Mild"
    },
    {
        "note": "Bilateral resting tremor, more pronounced on left. Moderate bradykinesia affecting both hands. Rigidity present in all limbs. Patient exhibits masked facies. Gait is shuffling with reduced arm swing. Mild postural instability on pull test.",
        "updrs_motor": 28,
        "severity": "Moderate"
    },
    {
        "note": "Severe bilateral tremor affecting hands and jaw. Marked bradykinesia - patient has difficulty initiating movements. Severe rigidity in all extremities. Postural instability present - patient required assistance. Freezing of gait observed.",
        "updrs_motor": 52,
        "severity": "Severe"
    },
    {
        "note": "Minimal right hand tremor only when stressed. No rigidity detected. Finger tapping speed normal. Gait entirely normal. No balance issues. Patient reports symptoms are very mild and only occasional.",
        "updrs_motor": 4,
        "severity": "Very Mild"
    },
    {
        "note": "Patient shows pronounced tremor in both hands and head. Severe bradykinesia - takes several seconds to initiate any movement. Wheelchair-bound due to severe postural instability. Speech is hypophonic and difficult to understand. Requires full assistance with daily activities.",
        "updrs_motor": 68,
        "severity": "Advanced"
    }
]

print(f"✅ Created {len(synthetic_data)} synthetic clinical notes")
print("\n📋 Example note:")
print(f"Severity: {synthetic_data[0]['severity']}")
print(f"Note: {synthetic_data[0]['note'][:120]}...")
print(f"True UPDRS Score: {synthetic_data[0]['updrs_motor']}")

✅ Created 5 synthetic clinical notes

📋 Example note:
Severity: Mild
Note: Patient presents with mild resting tremor in right hand, amplitude approximately 1cm. No tremor in left hand. Slight rig...
True UPDRS Score: 8


In [16]:
import re

def predict_updrs(clinical_note):
    """
    Rule-based UPDRS prediction (Day 1 baseline)
    
    This is a temporary baseline while we debug MedGemma.
    Uses keyword matching to estimate UPDRS scores.
    """
    note_lower = clinical_note.lower()
    score = 0
    
    # Severity multipliers
    if "very severe" in note_lower or "wheelchair" in note_lower or "assistance" in note_lower:
        base_multiplier = 3.5
    elif "severe" in note_lower or "marked" in note_lower:
        base_multiplier = 2.5
    elif "moderate" in note_lower:
        base_multiplier = 1.8
    elif "mild" in note_lower:
        base_multiplier = 1.2
    else:
        base_multiplier = 1.0
    
    # Symptom scoring
    symptom_score = 0
    
    # Tremor
    if "bilateral tremor" in note_lower:
        symptom_score += 12
    elif "tremor" in note_lower:
        symptom_score += 6
    
    # Rigidity
    if "severe rigidity" in note_lower or "rigidity in all" in note_lower:
        symptom_score += 12
    elif "rigidity" in note_lower:
        symptom_score += 6
    
    # Bradykinesia
    if "severe bradykinesia" in note_lower or "marked bradykinesia" in note_lower:
        symptom_score += 12
    elif "bradykinesia" in note_lower or "slowing" in note_lower:
        symptom_score += 6
    
    # Postural instability / Gait
    if "postural instability" in note_lower or "balance" in note_lower:
        symptom_score += 8
    if "shuffling" in note_lower or "freezing" in note_lower:
        symptom_score += 6
    
    # Speech/Face
    if "masked facies" in note_lower:
        symptom_score += 4
    if "hypophonic" in note_lower or "speech" in note_lower:
        symptom_score += 4
    
    # Calculate final score
    score = int(symptom_score * base_multiplier)
    
    # Clamp to valid range
    score = max(0, min(108, score))
    
    return score

print("✅ Rule-based UPDRS prediction created (Day 1 baseline)")
print("📝 Note: This is a temporary baseline. We'll improve it with MedGemma on Day 2!")

✅ Rule-based UPDRS prediction created (Day 1 baseline)
📝 Note: This is a temporary baseline. We'll improve it with MedGemma on Day 2!


In [17]:
# Test on moderate case
test_case = synthetic_data[1]
test_note = test_case["note"]
true_score = test_case["updrs_motor"]

print("🔄 Making prediction with rule-based baseline...\n")
predicted_score = predict_updrs(test_note)

print("="*70)
print("📋 Clinical Note:")
print("="*70)
print(test_note)
print("\n" + "="*70)
print("📊 RESULTS:")
print("="*70)
print(f"🎯 True UPDRS Score: {true_score}")
print(f"🤖 Predicted Score: {predicted_score}")
error = abs(predicted_score - true_score)
print(f"❌ Absolute Error: {error}")
accuracy = max(0, 100 - (error/true_score)*100)
print(f"✅ Relative Accuracy: {accuracy:.1f}%")
print("="*70)

🔄 Making prediction with rule-based baseline...

📋 Clinical Note:
Bilateral resting tremor, more pronounced on left. Moderate bradykinesia affecting both hands. Rigidity present in all limbs. Patient exhibits masked facies. Gait is shuffling with reduced arm swing. Mild postural instability on pull test.

📊 RESULTS:
🎯 True UPDRS Score: 28
🤖 Predicted Score: 64
❌ Absolute Error: 36
✅ Relative Accuracy: 0.0%


In [18]:
import numpy as np

results = []

print("🔄 Evaluating rule-based baseline on all cases...\n")
print("="*70)

for i, case in enumerate(synthetic_data):
    predicted = predict_updrs(case["note"])
    true = case["updrs_motor"]
    error = abs(predicted - true)
    
    results.append({
        "case": i+1,
        "severity": case["severity"],
        "true": true,
        "predicted": predicted,
        "error": error
    })
    
    print(f"Case {i+1} - {case['severity']:12} | True: {true:3} | Predicted: {predicted:3} | Error: {error:2}")

# Calculate metrics
errors = [r["error"] for r in results]
mae = np.mean(errors)
max_error = np.max(errors)
min_error = np.min(errors)

print("\n" + "="*70)
print("📊 DAY 1 BASELINE PERFORMANCE (Rule-Based)")
print("="*70)
print(f"✅ Success Rate: {len(errors)}/{len(synthetic_data)} predictions")
print(f"📈 Mean Absolute Error (MAE): {mae:.1f} points")
print(f"📉 Best Case Error: {min_error} points")
print(f"📊 Worst Case Error: {max_error} points")
print("="*70)

baseline_mae = mae
print(f"\n🎯 DAY 1 BASELINE: MAE = {mae:.1f}")
print("📝 This is your starting point! Tomorrow we'll improve it!")

🔄 Evaluating rule-based baseline on all cases...

Case 1 - Mild         | True:   8 | Predicted:  31 | Error: 23
Case 2 - Moderate     | True:  28 | Predicted:  64 | Error: 36
Case 3 - Severe       | True:  52 | Predicted: 108 | Error: 56
Case 4 - Very Mild    | True:   4 | Predicted:  24 | Error: 20
Case 5 - Advanced     | True:  68 | Predicted: 105 | Error: 37

📊 DAY 1 BASELINE PERFORMANCE (Rule-Based)
✅ Success Rate: 5/5 predictions
📈 Mean Absolute Error (MAE): 34.4 points
📉 Best Case Error: 20 points
📊 Worst Case Error: 56 points

🎯 DAY 1 BASELINE: MAE = 34.4
📝 This is your starting point! Tomorrow we'll improve it!


In [20]:
def analyze_clinical_note(note):
    """
    Main function: Analyze clinical note and return UPDRS prediction
    """
    score = predict_updrs(note)
    
    # Determine severity
    if score <= 20:
        severity = "Mild"
    elif score <= 40:
        severity = "Moderate"
    elif score <= 60:
        severity = "Severe"
    else:
        severity = "Very Severe"
    
    return {
        "updrs_score": score,
        "severity": severity,
        "model_version": "ParkPredict v0.1 - Rule-Based Baseline"
    }

# Test it
print("🧪 Testing wrapper function...\n")
result = analyze_clinical_note(synthetic_data[2]["note"])

print("📊 Analysis Result:")
print(f"   UPDRS Score: {result['updrs_score']}")
print(f"   Severity: {result['severity']}")
print(f"   Model: {result['model_version']}")
print("\n✅ analyze_clinical_note() function ready!")


🧪 Testing wrapper function...

📊 Analysis Result:
   UPDRS Score: 108
   Severity: Very Severe
   Model: ParkPredict v0.1 - Rule-Based Baseline

✅ analyze_clinical_note() function ready!
